In [1]:
import time
import numpy as np
import h5py
import matplotlib.pyplot as plt
from PIL import Image
from dnn_app_utils_v3 import *
import torch
import torchvision
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

ModuleNotFoundError: No module named 'dnn_app_utils_v3'

In [ ]:
train_x_orig, train_y, test_x_orig, test_y, classes = load_data()

In [ ]:
index = 7
plt.imshow(train_x_orig[index])
print ("y = " + str(train_y[0,index]) + ". It's a " + classes[train_y[0,index]].decode("utf-8") +  " picture.")

In [ ]:
# Reshape the training and test examples 
train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T   # The "-1" makes reshape flatten the remaining dimensions
test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T

# Standardize data to have feature values between 0 and 1.
train_x = train_x_flatten/255.
test_x = test_x_flatten/255.

print ("train_x's shape: " + str(train_x.shape))
print ("test_x's shape: " + str(test_x.shape))

In [ ]:
train_x = torch.tensor(train_x.T, dtype=torch.float32)
train_y = torch.tensor(train_y.T, dtype=torch.float32)
print ("train_x's shape: " + str(train_x.shape))
print ("train_y's shape: " + str(train_y.shape))

test_x = torch.tensor(test_x.T, dtype=torch.float32)
print ("test_x's shape: " + str(test_x.shape))


In [ ]:
train_ds = TensorDataset(train_x, train_y)
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)

In [ ]:
model = torch.nn.Sequential(
    # 1
    torch.nn.Linear(12288, 20), 
    torch.nn.ReLU(),
    # 2
    torch.nn.Linear(20, 7), 
    torch.nn.ReLU(),
    # 3
    torch.nn.Linear(7, 5), 
    torch.nn.ReLU(),
    # 4
    torch.nn.Linear(5, 1), 
    torch.nn.Sigmoid()
    )

In [ ]:
creterion = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
def train(epochs):
    for epoch in range (epochs):
        for x, y in train_dl:
            output = model(x)
            loss = creterion(output, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if epoch % 100 == 0:
            print(epoch, loss.item())

In [ ]:
train(2500) # число итераций обучения

In [ ]:
torch.save(model, "model.pt")

In [ ]:
model = torch.load("model.pt")
model.eval()

In [ ]:
def predict(self, test_images):
    self.eval()
    
    count = test_images.shape[0]
    result_np = []
        
    for idx in range(0, count):
        pred = self(test_images[idx])
        pred_np = pred.detach().numpy()
        for elem in pred_np:
            result_np.append(elem)
    return result_np

In [ ]:
res = predict(model, test_x)
print(len(res))
print(res)